In [1]:
import stanza
from pymongo import MongoClient
from pprint import pprint
import re

In [32]:
uri = open('./../../mongo_uri.txt', 'r', encoding='utf-8').read()
client = MongoClient(uri)
print(client)
db = client.boletines_db

MongoClient(host=['clustershido-shard-00-00-uxkjr.gcp.mongodb.net:27017', 'clustershido-shard-00-01-uxkjr.gcp.mongodb.net:27017', 'clustershido-shard-00-02-uxkjr.gcp.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='ClusterShido-shard-0', ssl=True)


In [3]:
stanza.download('es', processors='tokenize,mwt,pos,ner')

2020-05-22 16:00:00 INFO: Downloading these customized packages for language: es (Spanish)...
| Processor       | Package  |
------------------------------
| tokenize        | ancora   |
| mwt             | ancora   |
| pos             | ancora   |
| ner             | conll02  |
| pretrain        | ancora   |
| backward_charlm | newswiki |
| forward_charlm  | newswiki |

2020-05-22 18:13:31 INFO: Finished downloading models and saved to C:\Users\Usuario.000\stanza_resources.


In [17]:
nlp = stanza.Pipeline('es', use_gpu=True, processors='tokenize,mwt')

2020-05-22 19:07:11 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |

2020-05-22 19:07:11 INFO: Use device: cpu
2020-05-22 19:07:11 INFO: Loading: tokenize
2020-05-22 19:07:11 INFO: Loading: mwt
2020-05-22 19:07:11 INFO: Done loading processors!


In [33]:
txt = db.boletines.find_one({'estado':'Jalisco'})['boletin']
doc = nlp(txt)

In [35]:
for i_sent, sent in enumerate(doc.sentences):
    print(f'-------- Oración {i_sent+1} -----------')
    for token in sent.tokens:
#         print(f'token: {token.text}\t\twords: {", ".join([word.text for word in token.words])}')
        print(f'token: {token.text}')
    print('------------------------------------\n')

-------- Oración 1 -----------
token: Andrés
token: Manuel
token: López
token: Obrador
token: López
token: Obrador
token: dio
token: un
token: voto
token: de
token: confianza
token: el
token: ofrecimiento
token: que
token: hizo
token: el
token: presidente
token: del
token: Instituto
token: Nacional
token: Electoral
token: ,
token: Lorenzo
token: Córdova
token: que
token: no
token: habrá
token: fraude
token: electoral
token: y
token: no
token: se
token: caerá
token: el
token: sistema
token: ,
token: “eso
token: es
token: lo
token: que
token: se
token: necesita
token: ,
token: que
token: no
token: vaya
token: a
token: suceder
token: lo
token: del
token: 2006
token: que
token: no
token: se
token: dieron
token: a
token: conocer
token: los
token: resultados
token: ,
token: porque
token: arreglaron
token: los
token: paquetes
token: electorales
token: para
token: imponer
token: a
token: Felipe
token: Calderón
token: ”
token: .
------------------------------------

-------- Oración 2 ---------

In [28]:
test = "En un pronunciamiento especial, el candidato de la coalición Todos por México condenó enérgicamente los abusos (suscitados a consecuencia de la llamada política de “Tolerancia Cero”), que incluyen la separación de niños y padres, así como el confinamiento de menores en jaulas. Meade llamó a la Organización de las Naciones Unidas y a Unicef a intervenir para detener los atropellos."
dt = nlp(test)

In [29]:
for i_sent, sent in enumerate(dt.sentences):
    print(f'-------- Oración {i_sent+1} -----------')
    for token in sent.tokens:
        print(f'token: {token.text}')
    print('------------------------------------\n')

-------- Oración 1 -----------
token: En
token: un
token: pronunciamiento
token: especial
token: ,
token: el
token: candidato
token: de
token: la
token: coalición
token: Todos
token: por
token: México
token: condenó
token: enérgicamente
token: los
token: abusos
token: (
token: suscitados
token: a
token: consecuencia
token: de
token: la
token: llamada
token: política
token: de
token: “
token: Tolerancia
token: Cero”
token: )
token: ,
token: que
token: incluyen
token: la
token: separación
token: de
token: niños
token: y
token: padres
token: ,
token: así
token: como
token: el
token: confinamiento
token: de
token: menores
token: en
token: jaulas
token: .
------------------------------------

-------- Oración 2 -----------
token: Meade
token: llamó
token: a
token: la
token: Organización
token: de
token: las
token: Naciones
token: Unidas
token: y
token: a
token: Unicef
token: a
token: intervenir
token: para
token: detener
token: los
token: atropellos
token: .
--------------------------------

In [36]:
client.close()